# Wan 2.2 Distributed Inference Simulation

This notebook simulates the execution of a Wan 2.2 Video Generation workflow across a logical pool of heterogeneous GPUs (Kaggle T4s, Colab L4s).

It demonstrates:
1.  **VRAM-aware Scheduling**: Assigning tasks to workers that fit.
2.  **FSDP / Model Sharding**: Automatically splitting large models (FP16) across multiple workers when they exceed single-GPU capacity.
3.  **Pipeline Parallelism**: Distributing the graph (TextEnc -> UNet -> VAE) across the pool.

In [ ]:
import asyncio
import json
from wan_pool.controller import Controller
from wan_pool.worker import Worker, WorkerType, DeviceType

# 1. Setup the Logical Cluster
# We mimic a typical distributed setup: 2x T4 (Kaggle), 1x L4 (Colab), 1x CPU (Local)

workers = [
    Worker(worker_id="w1", name="Kaggle_T4_A", w_type=WorkerType.KAGGLE, vram_gb=15.0),
    Worker(worker_id="w2", name="Kaggle_T4_B", w_type=WorkerType.KAGGLE, vram_gb=15.0),
    Worker(worker_id="w3", name="Colab_L4",    w_type=WorkerType.COLAB,  vram_gb=22.0),
    Worker(worker_id="w4", name="Local_CPU",   w_type=WorkerType.LOCAL,  vram_gb=32.0, device=DeviceType.CPU)
]

controller = Controller(workers)
print("Cluster initialized.")

In [ ]:
# 2. Load the ComfyUI Workflow
# We use the JSON provided in the prompt (Wan 2.2 14B I2V)
# Note: We will modify the model filenames in the next steps to trigger different behaviors.

workflow_template = {
  "id": "job_wan_14b",
  "nodes": [
    {
      "id": 1, "type": "CLIPTextEncode",
      "widgets_values": ["a cinematic shot of a robot"],
      "inputs": []
    },
    {
      "id": 2, "type": "UNETLoader",
      "widgets_values": ["wan2.2_14b_fp8.safetensors"]
    },
    {
      "id": 3, "type": "ModelSampling",
      "inputs": [{"name": "model", "link": 20}]
    },
    {
      "id": 4, "type": "KSampler",
      "inputs": [
          {"name": "model", "link": 30},
          {"name": "positive", "link": 10}
      ]
    },
    {
      "id": 5, "type": "VAELoader",
      "widgets_values": ["wan_vae.safetensors"]
    },
    {
      "id": 6, "type": "VAEDecode",
      "inputs": [
          {"name": "samples", "link": 40},
          {"name": "vae", "link": 50}
      ]
    }
  ],
  "links": [
    [10, 1, 0, 4, 1, "CONDITIONING"],
    [20, 2, 0, 3, 0, "MODEL"],
    [30, 3, 0, 4, 0, "MODEL"],
    [40, 4, 0, 6, 0, "LATENT"],
    [50, 5, 0, 6, 1, "VAE"]
  ]
}


## Scenario A: FP8 Inference (Standard)
The model is `wan2.2_14b_fp8.safetensors`. This is approximately **14GB**.
- **Kaggle T4 (15GB)**: Fits (tightly).
- **Colab L4 (22GB)**: Fits comfortably.

Expected Behavior: The Planner should assign the UNet to one of the T4s or the L4.

In [ ]:
import copy
job_fp8 = copy.deepcopy(workflow_template)
# Ensure filename implies FP8
job_fp8["nodes"][1]["widgets_values"] = ["wan2.2_14b_fp8.safetensors"]

controller.submit_job(job_fp8)
await controller.run()

## Scenario B: FP16 Inference (Oversized / FSDP)
The model is `wan2.2_14b_fp16.safetensors`. This is approximately **28GB**.
- **Kaggle T4 (15GB)**: Too small.
- **Colab L4 (22GB)**: Too small.

Expected Behavior: The Planner must detect that no single worker fits 28GB. It should form a **Virtual Worker Group** (e.g., T4_A + T4_B = 30GB, or T4 + L4) and split the task.

In [ ]:
job_fp16 = copy.deepcopy(workflow_template)
# Rename to FP16 to trigger size estimation of ~28GB
job_fp16["nodes"][1]["widgets_values"] = ["wan2.2_14b_fp16.safetensors"]

controller.submit_job(job_fp16)
await controller.run()